## Interpretation

---

The interpretation of the weights in logistic regression differs from the interpretation of the weights in linear regression, since the outcome in logistic regression is a probability between 0 and 1. The weights do not influence the probability linearly any longer. The weighted sum is transformed by the logistic function to a probability. Therefore we need to reformulate the equation for the interpretation so that only the linear term is on the right side of the formula. To be explicity, let's use $\mathbb{P}(Y_i = 1) = \pi_i$:

$$
log\left(\frac{\mathbb{P}(y=1)}{1-\mathbb{P}(y=1)}\right)=log\left(\frac{\mathbb{P}(y=1)}{\mathbb{P}(y=0)}\right)=\beta_{0}+\beta_{1}x_{1}+\ldots+\beta_{p}x_{p}
$$

Exponentiating both sides leads us to the odds:

$$
\frac{\mathbb{P}(y=1)}{1-\mathbb{P}(y=1)}=odds=exp\left(\beta_{0}+\beta_{1}x_{1}+\ldots+\beta_{p}x_{p}\right)
$$

Then we compare what happens when we increase one of the feature values by 1. But instead of looking at the difference, we look at the ratio of the two predictions:

$$
\frac{odds_{x_j+1}}{odds}=\frac{exp\left(\beta_{0}+\beta_{1}x_{1}+\ldots+\beta_{j}(x_{j}+1)+\ldots+\beta_{p}x_{p}\right)}{exp\left(\beta_{0}+\beta_{1}x_{1}+\ldots+\beta_{j}x_{j}+\ldots+\beta_{p}x_{p}\right)}
$$

We apply the following rule:

$$
\frac{exp(a)}{exp(b)}=exp(a-b)
$$

And we remove many terms:

$$
\frac{odds_{x_j+1}}{odds}=exp\left(\beta_{j}(x_{j}+1)-\beta_{j}x_{j}\right)=exp\left(\beta_j\right)
$$

In the end, we have something as simple as exp() of a feature weight. A change in a feature by one unit changes the odds ratio (multiplicative) by a factor of exp(βj). We could also interpret it this way: A change in xj by one unit increases the log odds ratio by the value of the corresponding weight.

Discussion:

* How do we interpret the intercept?

## Application

---

In [1]:
#Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import scipy.stats as stats

In [2]:
#Import Dataset
dataset = pd.read_csv('social_network_data.csv')
dataset.describe()

,User ID,Age,EstimatedSalary,Purchased
count,4.000000e+02,400.000000,400.000000,400.000000
mean,1.569154e+07,37.655000,69742.500000,0.357500
std,7.165832e+04,10.482877,34096.960282,0.479864
min,1.556669e+07,18.000000,15000.000000,0.000000
25%,1.562676e+07,29.750000,43000.000000,0.000000
50%,1.569434e+07,37.000000,70000.000000,0.000000
75%,1.575036e+07,46.000000,88000.000000,1.000000
max,1.581524e+07,60.000000,150000.000000,1.000000


In [3]:
dataset.rename(columns={'User ID': 'id', 'Age': 'age', 'EstimatedSalary': 'est_salary', 'Purchased': 'purchased'},
               inplace=True)

In [4]:
x = dataset.iloc[:,[2,3]].values
y = dataset.iloc[:,4].values

In [5]:
classifier1 = LogisticRegression()
classifier1.fit(x, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [6]:
print('intercept:', classifier1.intercept_)
print('coefficient:', classifier1.coef_)

intercept: [-0.00510052]
coefficient: [[-5.88775639e-03 -9.74009872e-09]]


In [7]:
#Split Training Set and Testing Set
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.5)

In [8]:
#Feature Scaling
sc_X=StandardScaler()
x_train=sc_X.fit_transform(x_train)
x_test=sc_X.transform(x_test)

In [9]:
#Training the Logistic Model
classifier = LogisticRegression()
classifier.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [10]:
print('intercept:', classifier.intercept_)
print('coefficient:', classifier.coef_)

intercept: [-0.81452461]
coefficient: [[2.11907143 1.17728586]]


---
## Assignment

**Question 8** (Assignment): Perform gradient descent or Newton Raphson to find the Maximum Likelihood Estimates (MLE).  

_Note_: I use all observations instead of splitting the dataset into training and test sets, since there is no clear need for doing so here.  

\begin{equation}
\text{With } \frac{\delta\mathcal{l}(\beta)}{\delta\beta} = X^T(y-p) \,\text{ and } \frac{\delta^2\mathcal{l}(\beta)}{\delta\beta\delta\beta^T} = -X^TWX \text{, we have}\\
\beta^{new} = \beta^{old} + (X^TWX)^{-1}X^T(y-p)
\end{equation}

In [11]:
def probability(x, beta):
    odds = np.exp(np.matmul(x, beta))
    return (odds / (1 + odds))

In [12]:
def gradient(x, y, prob):
    return np.matmul(x.transpose(), y - prob)

In [13]:
def hessian(x, prob):
    return (-np.matmul(np.matmul(x.transpose(), np.diag(prob * (1 - prob))), x))

In [14]:
def update_beta(x, y, beta):
    prob = probability(x, beta)
    return (beta - np.matmul(np.linalg.inv(hessian(x, prob)), gradient(x, y, prob)))

In [ ]:
x_std = np.hstack((np.ones((x.shape[0], 1)), StandardScaler().fit_transform(x)))

In [15]:
beta = np.array([0, 0, 0])
tol = 1e-6
beta_next = update_beta(x_std, y, beta)
iters = 1
while np.sum(abs(beta_next - beta)) > tol:
    beta = beta_next
    beta_next = update_beta(x_std, y, beta)
    if iters % 100 == 0:
        print('Iteration %d. MLE of beta:' % iters)
        print(beta)
beta = beta_next
print(beta)

[-1.13812197  2.44457954  1.22258176]


**Question 9**: What is the impact on the odds of a purchase with a dollar increase in EstimatedSalary?

In [16]:
np.exp(beta[2] / np.std(x, axis=0)[1])

1.000035901582615

beta[2] (1.222582) gives us the odds of a purchase with a sample standard deviation increase in estimated salary relative to the original odds. Divided by the sample standard deviation, it gives us the relative odds (of a purchase) with a dollar increase, which is 1.000036.

**Question 10**: What is the odds of purchase with an age of 38 and estimated salary of $60,000?

In [93]:
age_std = (38 - means[0]) / stdevs[0]
salary_std = (60000 - means[1]) / stdevs[1]
np.exp(beta[0] + beta[1]*age_std + beta[2]*salary_std)

0.24479528277715504

The odds of purchase are 0.244795, i.e. the individual is about 4 times as likely to not purchase as to purchase.

## Diagnostics & Prediction

---

As we are unable to work with the residuals, as you would with OLS (we are using a binomial link function), a way to analyze classification accuracy and fit is through the _confusion matrix_.

**Question 11 (for my own curiosity)**: What are the performance metrics of the predictions obtained by fitting the model using the Newton-Raphson algorithm as above?

In the absence of further information, a reasonable criterion for predicting a positive outcome is when the predicted probability exceeds 0.5.

In [30]:
cm = confusion_matrix(y, (probability(x_std, beta) > 0.5).astype(int))

In [39]:
print('sensitivity = %f, specificity = %f' % (cm[1,1] / cm[1].sum(), cm[0,0] / (cm[0,0] + cm[1,0])))

sensitivity = 0.713287, specificity = 0.851986


This model has a sensitivity of {{ cm[0, 0] / cm[0].sum() }}